In [494]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import metrics
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import pickle

In [495]:
df=pd.read_csv("weatherAUS.csv")

In [496]:
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [497]:
df.tail()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
145455,2017-06-21,Uluru,2.8,23.4,0.0,NaN,NaN,E,31.0,SE,...,51.0,24.0,1024.6,1020.3,NaN,NaN,10.1,22.4,No,No
145456,2017-06-22,Uluru,3.6,25.3,0.0,NaN,NaN,NNW,22.0,SE,...,56.0,21.0,1023.5,1019.1,NaN,NaN,10.9,24.5,No,No
145457,2017-06-23,Uluru,5.4,26.9,0.0,NaN,NaN,N,37.0,SE,...,53.0,24.0,1021.0,1016.8,NaN,NaN,12.5,26.1,No,No
145458,2017-06-24,Uluru,7.8,27.0,0.0,NaN,NaN,SE,28.0,SSE,...,51.0,24.0,1019.4,1016.5,3.0,2.0,15.1,26.0,No,No
145459,2017-06-25,Uluru,14.9,NaN,0.0,NaN,NaN,NaN,NaN,ESE,...,62.0,36.0,1020.2,1017.9,8.0,8.0,15.0,20.9,No,NaN


In [498]:
df.columns

Index(['Date', 'Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation',
       'Sunshine', 'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm',
       'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
       'Temp3pm', 'RainToday', 'RainTomorrow'],
      dtype='object')

In [499]:
df.dtypes

Date              object
Location          object
MinTemp          float64
MaxTemp          float64
Rainfall         float64
Evaporation      float64
Sunshine         float64
WindGustDir       object
WindGustSpeed    float64
WindDir9am        object
WindDir3pm        object
WindSpeed9am     float64
WindSpeed3pm     float64
Humidity9am      float64
Humidity3pm      float64
Pressure9am      float64
Pressure3pm      float64
Cloud9am         float64
Cloud3pm         float64
Temp9am          float64
Temp3pm          float64
RainToday         object
RainTomorrow      object
dtype: object

In [500]:
df.shape

(145460, 23)

In [501]:
df.isnull().sum()

Date                 0
Location             0
MinTemp           1485
MaxTemp           1261
Rainfall          3261
Evaporation      62790
Sunshine         69835
WindGustDir      10326
WindGustSpeed    10263
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am      1767
WindSpeed3pm      3062
Humidity9am       2654
Humidity3pm       4507
Pressure9am      15065
Pressure3pm      15028
Cloud9am         55888
Cloud3pm         59358
Temp9am           1767
Temp3pm           3609
RainToday         3261
RainTomorrow      3267
dtype: int64

In [502]:
num=[feature for feature in df.columns if df[feature].dtypes!="0"]
discrete=[feature for feature in num if len(df[feature].unique())<=25]
cont=[feature for feature in num if feature not in discrete]
categorical=[feature for feature in df.columns if feature not in num]

In [503]:
def RandomSampleImputation(df,feature):
  df[feature]=df[feature]
  random_sample=df[feature].dropna().sample(df[feature].isnull().sum(),random_state=0)
  random_sample.index=df[df[feature].isnull()].index
  df.loc[df[feature].isnull(),feature]=random_sample

In [504]:
df["Pressure3pm"].fillna(df["Pressure3pm"].mean(), inplace=True)

In [505]:
df["Pressure9am"].fillna(df["Pressure9am"].mean(), inplace=True)

In [506]:
df["MinTemp"].fillna(df["MinTemp"].mean(), inplace=True)

In [507]:
df["MaxTemp"].fillna(df["MaxTemp"].mean(), inplace=True)

In [508]:
df["Rainfall"].fillna(df["Rainfall"].mean(), inplace=True)

In [509]:
df["WindGustSpeed"].fillna(df["WindGustSpeed"].mean(), inplace=True)

In [510]:
df["WindSpeed9am"].fillna(df["WindSpeed9am"].mean(), inplace=True)

In [511]:
df["WindSpeed3pm"].fillna(df["WindSpeed3pm"].mean(), inplace=True)

In [512]:
df["Pressure9am"].fillna(df["Pressure9am"].mean(), inplace=True)

In [513]:
df["Humidity9am"].fillna(df["Humidity9am"].mean(), inplace=True)

In [514]:
df["Humidity3pm"].fillna(df["Humidity3pm"].mean(), inplace=True)

In [515]:
df["Temp3pm"].fillna(df["Temp3pm"].mean(), inplace=True)

In [516]:
df["Temp9am"].fillna(df["Temp9am"].mean(), inplace=True)

In [517]:
RandomSampleImputation(df,"Cloud9am")

In [518]:
RandomSampleImputation(df,"Evaporation")

In [519]:
RandomSampleImputation(df,"Cloud3pm")

In [520]:
RandomSampleImputation(df,"Sunshine")

In [521]:
df.corr()

<ipython-input-521-2f6f6606aa2c>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()


,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm
MinTemp,1.000000,0.733400,0.102706,0.263250,0.034775,0.172553,0.173404,0.173058,-0.230970,0.005995,-0.423584,-0.433147,0.049293,0.014844,0.897692,0.699211
MaxTemp,0.733400,1.000000,-0.074040,0.328974,0.233493,0.065895,0.014294,0.049717,-0.497927,-0.498760,-0.308309,-0.396622,-0.174609,-0.161461,0.879170,0.968713
Rainfall,0.102706,-0.074040,1.000000,-0.028580,-0.125236,0.126446,0.085925,0.056527,0.221380,0.248905,-0.159055,-0.119541,0.133378,0.110850,0.011069,-0.077684
Evaporation,0.263250,0.328974,-0.028580,1.000000,0.152067,0.112185,0.107306,0.070449,-0.281094,-0.215943,-0.160841,-0.175323,-0.085556,-0.083914,0.310475,0.319016
Sunshine,0.034775,0.233493,-0.125236,0.152067,1.000000,-0.018850,0.002723,0.024119,-0.248893,-0.317475,0.023789,-0.009485,-0.299892,-0.309680,0.146215,0.246210
WindGustSpeed,0.172553,0.065895,0.126446,0.112185,-0.018850,1.000000,0.577319,0.657243,-0.207964,-0.025355,-0.425760,-0.383938,0.042596,0.060980,0.145904,0.031884
WindSpeed9am,0.173404,0.014294,0.085925,0.107306,0.002723,0.577319,1.000000,0.512427,-0.268271,-0.030887,-0.215339,-0.165388,0.014044,0.032633,0.127592,0.004476
WindSpeed3pm,0.173058,0.049717,0.056527,0.070449,0.024119,0.657243,0.512427,1.000000,-0.143458,0.016275,-0.277604,-0.239659,0.034715,0.013838,0.161060,0.027587
Humidity9am,-0.230970,-0.497927,0.221380,-0.281094,-0.248893,-0.207964,-0.268271,-0.143458,1.000000,0.659072,0.131503,0.176009,0.275274,0.209483,-0.469641,-0.490709
Humidity3pm,0.005995,-0.498760,0.248905,-0.215943,-0.317475,-0.025355,-0.030887,0.016275,0.659072,1.000000,-0.025848,0.048695,0.311540,0.312866,-0.216964,-0.555608


In [522]:
df["RainToday"]=pd.get_dummies(df["RainToday"],drop_first=True)
df["RainTomorrow"]=pd.get_dummies(df["RainTomorrow"],drop_first=True)

In [523]:
for feature in categorical:
  print(feature,(df.groupby([feature])["RainTomorrow"].mean().sort_values(ascending=False)).index)

In [524]:
label_encoder = LabelEncoder()
df['WindGustDir'] = label_encoder.fit_transform(df['WindGustDir'])
df['WindDir9am'] = label_encoder.fit_transform(df['WindDir9am'])
df['WindDir3pm'] = label_encoder.fit_transform(df['WindDir3pm'])
df['Location'] = label_encoder.fit_transform(df['Location'])

In [525]:
df['WindGustDir']

0         13
1         14
2         15
3          4
4         13
          ..
145455     0
145456     6
145457     3
145458     9
145459    16
Name: WindGustDir, Length: 145460, dtype: int64

In [526]:
df["WindGustDir"].fillna(df["WindGustDir"].mode(), inplace=True)

In [527]:
df["WindDir9am"].fillna(df["WindDir9am"].mode(), inplace=True)
df["WindDir3pm"].fillna(df["WindDir3pm"].mode(), inplace=True)


In [528]:
df.isnull().sum()

Date             0
Location         0
MinTemp          0
MaxTemp          0
Rainfall         0
Evaporation      0
Sunshine         0
WindGustDir      0
WindGustSpeed    0
WindDir9am       0
WindDir3pm       0
WindSpeed9am     0
WindSpeed3pm     0
Humidity9am      0
Humidity3pm      0
Pressure9am      0
Pressure3pm      0
Cloud9am         0
Cloud3pm         0
Temp9am          0
Temp3pm          0
RainToday        0
RainTomorrow     0
dtype: int64

In [529]:
date_format = '%Y-%m-%d'
df['Date'] = pd.to_datetime(df['Date'], format=date_format)
print(df)

             Date  Location  MinTemp    MaxTemp  Rainfall  Evaporation  \
0      2008-12-01         2     13.4  22.900000       0.6          2.4   
1      2008-12-02         2      7.4  25.100000       0.0          3.6   
2      2008-12-03         2     12.9  25.700000       0.0          2.6   
3      2008-12-04         2      9.2  28.000000       0.0         18.4   
4      2008-12-05         2     17.5  32.300000       1.0          5.4   
...           ...       ...      ...        ...       ...          ...   
145455 2017-06-21        41      2.8  23.400000       0.0          1.4   
145456 2017-06-22        41      3.6  25.300000       0.0          7.6   
145457 2017-06-23        41      5.4  26.900000       0.0          6.8   
145458 2017-06-24        41      7.8  27.000000       0.0          2.6   
145459 2017-06-25        41     14.9  23.221348       0.0          1.4   

        Sunshine  WindGustDir  WindGustSpeed  WindDir9am  ...  Humidity9am  \
0            8.3           13    

In [530]:
df=df.drop(labels="Date",axis=1)

In [531]:
x=df.drop(["RainTomorrow"],axis=1)
y=df["RainTomorrow"]

In [532]:
sc=StandardScaler()
sc.fit(x)
X=sc.transform(x)
x=pd.DataFrame(X,columns=x.columns)

In [533]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [534]:
model=XGBClassifier()
model.fit(x_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [535]:
y_pred=model.predict(x_test)

In [536]:
accuracy_score(y_test,y_pred)

0.8575553416746872

In [537]:
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.95      0.91     22726
           1       0.74      0.53      0.62      6366

    accuracy                           0.86     29092
   macro avg       0.81      0.74      0.77     29092
weighted avg       0.85      0.86      0.85     29092



In [538]:
booster = model.get_booster()
model_summary = booster.get_dump()
print("Model Summary:")
for i, tree in enumerate(model_summary):
    print(f"Tree {i + 1}:")
    print(tree)

Model Summary:
Tree 1:
0:[Humidity3pm<0.975075364] yes=1,no=2,missing=1
	1:[Humidity3pm<-0.0131677184] yes=3,no=4,missing=3
		3:[WindGustSpeed<1.25510812] yes=7,no=8,missing=7
			7:[Pressure3pm<-0.616134405] yes=15,no=16,missing=15
				15:[Humidity3pm<-1.12544489] yes=31,no=32,missing=31
					31:[Humidity3pm<-1.61393797] yes=63,no=64,missing=63
						63:leaf=-0.568796873
						64:leaf=-0.5115453
					32:[Sunshine<0.301206559] yes=65,no=66,missing=65
						65:leaf=-0.313571155
						66:leaf=-0.476363659
				16:[Sunshine<0.0370734259] yes=33,no=34,missing=33
					33:[WindGustSpeed<0.378464043] yes=67,no=68,missing=67
						67:leaf=-0.522473276
						68:leaf=-0.433999032
					34:[Rainfall<-0.192177296] yes=69,no=70,missing=69
						69:leaf=-0.564299703
						70:leaf=-0.506327033
			8:[Sunshine<0.06348674] yes=17,no=18,missing=17
				17:[WindDir3pm<-0.105977394] yes=35,no=36,missing=35
					35:[Humidity3pm<-1.6627872] yes=71,no=72,missing=71
						71:leaf=-0.373584926
						72:leaf=-0.0618

In [539]:
file=open("xgbclassifer.pkl","wb")
pickle.dump(model,file)